In [1]:
positions = [
    'pours',
    'contres',
    'abstentions'
]

In [2]:
import json
import pandas as pd
from glob import glob
import tqdm

In [3]:
def get_scrutin(filename):
    return json.load(open(filename))

scrutins = [ get_scrutin(f) for f in tqdm.tqdm(glob('data/**/VTAN*.json')) ]

len(scrutins)

100%|███████████████████████████████████████████████| 1810/1810 [00:00<00:00, 7371.81it/s]


1810

In [4]:
def get_votes(scrutin):
    data = scrutin
    groupes = data['scrutin']['ventilationVotes']['organe']['groupes']['groupe']

    #print(scrutin)
    
    votes = [ (p, g['organeRef'], g['vote']['decompteNominatif'][p]['votant'], g['vote']['positionMajoritaire'])
           for g in groupes
           for p in positions
           if g['vote']['decompteNominatif'][p] != None
          ]

    df = (
        pd.concat([
            pd
            .DataFrame
            .from_records([v] if not isinstance(v, list) else v)
            .assign(
                position=p,
                organe=o,
                date=scrutin['scrutin']['dateScrutin'],
                scrutin=scrutin['scrutin']['uid'],
                groupe_majorite= gv
            )
            for (p, o, v, gv) in votes
        ])
        .assign(
            position = lambda df: df.position.replace({'pours': 'pour', 'contres': 'contre', 'abstentions': 'abstention'})
        )
    )

    return df

votes = pd.concat([ get_votes(scrutin) for scrutin in scrutins ])

votes

,acteurRef,mandatRef,parDelegation,position,organe,date,scrutin,groupe_majorite
0,PA643184,PM797142,false,contre,PO800538,2022-10-25,VTANR5L16V375,contre
1,PA340643,PM796335,false,contre,PO800538,2022-10-25,VTANR5L16V375,contre
2,PA2449,PM797046,false,contre,PO800538,2022-10-25,VTANR5L16V375,contre
3,PA793194,PM796200,false,contre,PO800538,2022-10-25,VTANR5L16V375,contre
4,PA331835,PM796518,false,contre,PO800538,2022-10-25,VTANR5L16V375,contre
...,...,...,...,...,...,...,...,...
1,PA794008,PM796653,false,pour,PO800526,2022-12-15,VTANR5L16V805,pour
2,PA795362,PM797442,false,pour,PO800526,2022-12-15,VTANR5L16V805,pour
0,PA719138,PM796356,false,abstention,PO800532,2022-12-15,VTANR5L16V805,abstention
1,PA721046,PM797769,false,abstention,PO800532,2022-12-15,VTANR5L16V805,abstention


In [5]:
votes.to_csv('out/votes.csv', index=False)

In [6]:
votes.to_parquet('out/votes.par', index=False)

In [7]:
#scrutins[0]

In [8]:
scrutins_df = pd.DataFrame.from_records([
    {
        'uid': s['scrutin']['uid'],
        'objet': s['scrutin']['titre'],
        'type': s['scrutin']['typeVote']['libelleTypeVote'],
        'date': s['scrutin']['dateScrutin'],
        'seance': s['scrutin']['seanceRef'],
        'sort': s['scrutin']['sort']['code']
    }
    for s in scrutins
])

scrutins_df

,uid,objet,type,date,seance,sort
0,VTANR5L16V375,l'amendement n° 609 de M. Guedj après l'articl...,scrutin public ordinaire,2022-10-25,RUANR5L16S2023IDS26461,rejeté
1,VTANR5L16V791,le sous-amendement n° 3228 de M. Fournier à l'...,scrutin public ordinaire,2022-12-14,RUANR5L16S2023IDS26639,rejeté
2,VTANR5L16V67,l'amendement de suppression n° 213 de Mme Bath...,scrutin public ordinaire,2022-07-21,RUANR5L16S2022IDS26212,rejeté
3,VTANR5L16V1147,l'amendement n° 7 de Mme Garin et l'amendement...,scrutin public ordinaire,2023-03-08,RUANR5L16S2023IDS26863,rejeté
4,VTANR5L16V709,l'amendement n° 2886 de Mme Laernoes à l'artic...,scrutin public ordinaire,2022-12-08,RUANR5L16S2023IDS26604,rejeté
...,...,...,...,...,...,...
1805,VTANR5L16V1409,l'ensemble du projet de loi relatif aux jeux O...,scrutin public ordinaire,2023-04-11,RUANR5L16S2023IDS26997,adopté
1806,VTANR5L16V300,l'amendement n° 424 de M. Boucard après l'arti...,scrutin public ordinaire,2022-10-14,RUANR5L16S2023IDS26395,rejeté
1807,VTANR5L16V492,l'amendement n° 303 de M. Pancher à l'article ...,scrutin public ordinaire,2022-10-31,RUANR5L16S2023IDS26425,rejeté
1808,VTANR5L16V1376,le sous-amendement n° 107 de M. Meizonnet à l'...,scrutin public ordinaire,2023-04-06,RUANR5L16S2023IDS26966,rejeté


In [9]:
scrutins_df.type.value_counts()

scrutin public ordinaire    1775
scrutin public solennel       35
Name: type, dtype: int64

In [10]:
scrutins_df.to_csv('out/scrutins.csv', index=False)